# Merge the walking network and extracted crossings from osm, project sidewalk, traffic signs and curb heights

In [ ]:
# Select where to run notebook: "azure" or "local"
my_run = "azure"

In [ ]:
import geopandas as gpd
import pandas as pd

import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

In [ ]:
os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

In [ ]:
# Import network and crossings
CRS = st.CRS

gdf_network = gpd.read_file(cf.output_file_widths).to_crs(crs=CRS)
gdf_osm_crossings = gpd.read_file(cf.output_osm_crossings).to_crs(crs=CRS)
gdf_project_sidewalk_crossings = gpd.read_file(cf.output_project_sidewalk_crossings).to_crs(crs=CRS)
gdf_traffic_sign_crossings = gpd.read_file(cf.output_traffic_sign_crossings).to_crs(crs=CRS)

gdf_curb_crossings_4cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.04.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_6cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.06.gpkg').to_crs(crs=CRS)
gdf_curb_crossings_8cm = gpd.read_file(cf.output_curb_crossings_base + '_max_height_0.08.gpkg').to_crs(crs=CRS)

In [ ]:
# Set and delete columns
gdf_osm_crossings['crossing_type'] = 'osm'
gdf_project_sidewalk_crossings['crossing_type'] = 'project_sidewalk'
gdf_traffic_sign_crossings['crossing_type'] = 'traffic_sign'

gdf_curb_crossings_4cm['curb_height_max'] = 0.04
gdf_curb_crossings_6cm['curb_height_max'] = 0.06
gdf_curb_crossings_8cm['curb_height_max'] = 0.08

gdf_curb_crossings = pd.concat([gdf_curb_crossings_4cm, gdf_curb_crossings_6cm, gdf_curb_crossings_8cm])
gdf_curb_crossings['crossing_type'] = 'curb_height'

gdf_crossings = pd.concat([gdf_osm_crossings, gdf_project_sidewalk_crossings, 
                     gdf_traffic_sign_crossings, gdf_curb_crossings])
gdf_crossings = gdf_crossings.drop(columns=['index', 'element_type', 'osmid', 'crossing', 'highway', 'kerb',
       'tactile_paving', 'button_operated', 'traffic_signals:sound',
       'traffic_signals:vibration', 'crossing:island', 'crossing_ref',
       'bicycle', 'barrier', 'crossing:markings', 'nodes', 'footway',
       'surface', 'lit', 'smoothness', 'cycleway', 'mofa', 'moped', 'oneway',
       'sidewalk', 'traffic_sign', 'traffic_calming', 'natural'])

gdf_crossings['length'] = gdf_crossings.geometry.length

gdf_network['crossing'] = 'No'
gdf_crossings['crossing'] = 'Yes'

# Merge walking network and crossings
gdf_network_including_crossings = pd.concat([gdf_network, gdf_crossings])

# Export network including crossings
gdf_network_including_crossings.to_file(cf.output_basic_network_including_crossings, driver='GPKG')